In [14]:
me = !whoami
env = !which python
cuda = !whereis cuda  
cuda_version = !nvcc --version
print('User: ', me)
print('Conda env: ', env)
print('Cuda module: ', cuda)
print('Cuda version: ', cuda_version)

User:  ['pablorr']
Conda env:  ['/n/home01/pablorr/.conda/envs/torch37/bin/python']
Cuda module:  ['cuda:']
Cuda version:  ['nvcc: NVIDIA (R) Cuda compiler driver', 'Copyright (c) 2005-2017 NVIDIA Corporation', 'Built on Fri_Sep__1_21:08:03_CDT_2017', 'Cuda compilation tools, release 9.0, V9.0.176']
